In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy.dialects.postgresql import JSON, JSONB
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
import json
import csv
import datetime
from dateutil.parser import parser
import os
from pprint import pprint
from pathlib import Path
import glob
#from odo import odo
import importlib
import itertools
import sklearn.cluster
import pandas as pd
import numpy as np
import math
import operator
import sklearn
import matplotlib.pyplot as plt

#importlib.invalidate_caches()
%load_ext autoreload
%autoreload 2

In [2]:
audio_features = pd.read_pickle('../Pickle_Files/audio_features.pkl')
tracks = pd.read_pickle('../Pickle_Files/tracks.pkl')

#### Removing Mode and Time_Signature from Clustering as they have little variation ie: Mode is 0 or 1

In [3]:
audio_features['mode'].value_counts()

1    1481755
0     779960
Name: mode, dtype: int64

In [4]:
audio_features['time_signature'].value_counts()

4    1964293
3     229097
5      41751
1      23830
0       2744
Name: time_signature, dtype: int64

In [5]:
audio_features = audio_features.drop(['mode','time_signature'],axis=1)

In [6]:
audio_features = audio_features.set_index('track_uri')

In [7]:
audio_features.head(5)

,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,speechiness,tempo,valence
track_uri,,,,,,,,,,
spotify:track:0XLOf9LhyazPX9Ld8jPiUq,0.02040,0.708,0.610,0.000004,2,0.0641,-4.522,0.0574,123.038,0.305
spotify:track:7aF09WaavZAmAWuUeYxlYD,0.18300,0.593,0.868,0.000000,1,0.3650,-3.573,0.2950,104.988,0.496
spotify:track:0tKcYR2II1VCQWT79i5NrW,0.00683,0.600,0.810,0.210000,0,0.1550,-4.749,0.0479,167.880,0.298
spotify:track:6TWSVHx6z6E42JiwloGv1k,0.01620,0.503,0.918,0.024400,11,0.1860,-5.010,0.0464,140.000,0.418
spotify:track:5QqyRUZeBE04yJxsD1OC0I,0.01850,0.760,0.561,0.000019,1,0.1990,-8.697,0.1340,134.983,0.121


## K-Means Clustering

In [8]:
data = audio_features

In [9]:
# Convert DataFrame to matrix
mat = data.values
# Using sklearn
km = sklearn.cluster.KMeans(n_clusters=12)
km.fit(mat)
# Get cluster assignment labels
labels = km.labels_
# Format results as a DataFrame
average_song_cluster = pd.DataFrame(data=labels, columns=['cluster'], index=data.index)

In [10]:
average_song_cluster.head(5)

,cluster
track_uri,
spotify:track:0XLOf9LhyazPX9Ld8jPiUq,6
spotify:track:7aF09WaavZAmAWuUeYxlYD,8
spotify:track:0tKcYR2II1VCQWT79i5NrW,10
spotify:track:6TWSVHx6z6E42JiwloGv1k,1
spotify:track:5QqyRUZeBE04yJxsD1OC0I,1


### Saving for analysis

In [11]:
average_song_cluster.to_pickle('../Pickle_Files/clustered_songs.pkl')

#### Number of songs in each cluster

In [12]:
average_song_cluster['cluster'].value_counts()

6     354753
3     301046
2     299047
1     277806
8     261159
11    216820
10    172465
5     162875
7      82345
0      81250
4      49121
9       3028
Name: cluster, dtype: int64

#### Saving Cluster Centers

In [13]:
average_song_cluster_centroid = km.cluster_centers_

In [14]:
clustered = pd.merge(average_song_cluster,tracks,on='track_uri')

## Cluster Centers

In [15]:
from sklearn.metrics.pairwise import euclidean_distances

In [16]:
dists = euclidean_distances(average_song_cluster_centroid)

In [17]:
centroid_distance = pd.DataFrame(data=dists[:])

#### Finding closest clusters to each cluster

In [18]:
zero = centroid_distance[0].sort_values(ascending=True).head(4).index.values
one = centroid_distance[1].sort_values(ascending=True).head(4).index.values
two = centroid_distance[2].sort_values(ascending=True).head(4).index.values
three = centroid_distance[3].sort_values(ascending=True).head(4).index.values
four = centroid_distance[4].sort_values(ascending=True).head(4).index.values
five = centroid_distance[5].sort_values(ascending=True).head(4).index.values
six = centroid_distance[6].sort_values(ascending=True).head(4).index.values
seven = centroid_distance[7].sort_values(ascending=True).head(4).index.values
eight = centroid_distance[8].sort_values(ascending=True).head(4).index.values
nine = centroid_distance[9].sort_values(ascending=True).head(4).index.values
ten = centroid_distance[10].sort_values(ascending=True).head(4).index.values
eleven = centroid_distance[11].sort_values(ascending=True).head(4).index.values

zero = zero[1:4] 
one = one[1:4] 
two = two[1:4] 
three = three[1:4] 
four = four[1:4] 
five = five[1:4] 
six = six[1:4] 
seven = seven[1:4] 
eight = eight[1:4] 
nine = nine[1:4] 
ten = ten[1:4] 
eleven = eleven[1:4] 

centroid_distance['Closest'] = np.nan #Creating Closest Column

##Array of Arrays
arrays = [[np.array(zero)],[np.array(one)],[np.array(two)],[np.array(three)],[np.array(four)],[np.array(five)],
        [np.array(six)],[np.array(seven)],[np.array(eight)],[np.array(nine)],[np.array(ten)],[np.array(eleven)]]

In [19]:
Closest = pd.DataFrame(arrays)
centroid_distance['Closest'] = Closest
centroid_distance.columns =['zero', 'one','two', 'three','four', 'five','six', 'seven','eight',
                            'nine','ten','eleven','Closest']
centroid_distance.index =['zero', 'one','two', 'three','four', 'five','six', 'seven','eight',
                            'nine','ten','eleven']
centroid_distance['cluster'] = ['0','1','2','3','4','5','6','7','8','9','10','11']

In [20]:
centroid_distance

,zero,one,two,three,four,five,six,seven,eight,nine,ten,eleven,Closest,cluster
zero,0.000000,72.120132,51.145259,28.759939,126.368001,86.506566,60.893497,22.867903,40.119935,64.888267,104.802078,15.769685,"[11, 7, 3]",0
one,72.120132,0.000000,21.160250,45.279110,54.474092,14.393574,11.748461,49.984989,32.683643,136.446649,32.838627,59.641950,"[6, 5, 2]",1
two,51.145259,21.160250,0.000000,24.215421,75.633518,35.528187,9.755222,29.597512,11.573291,115.295337,53.998510,38.503370,"[6, 8, 1]",2
three,28.759939,45.279110,24.215421,0.000000,99.708075,59.554785,33.599893,13.936904,12.644897,91.474372,78.079589,14.470952,"[8, 7, 11]",3
four,126.368001,54.474092,75.633518,99.708075,0.000000,40.153912,66.108773,103.849462,87.147548,190.896559,21.635842,114.105781,"[10, 5, 1]",4
five,86.506566,14.393574,35.528187,59.554785,40.153912,0.000000,25.955963,64.284435,47.007652,150.822418,18.534722,73.961244,"[1, 10, 6]",5
six,60.893497,11.748461,9.755222,33.599893,66.108773,25.955963,0.000000,39.260545,21.071979,124.925694,44.481292,48.011928,"[2, 1, 8]",6
seven,22.867903,49.984989,29.597512,13.936904,103.849462,64.284435,39.260545,0.000000,19.975216,87.748585,82.372535,16.375316,"[3, 11, 8]",7
eight,40.119935,32.683643,11.573291,12.644897,87.147548,47.007652,21.071979,19.975216,0.000000,103.857734,65.512563,26.959601,"[2, 3, 7]",8
nine,64.888267,136.446649,115.295337,91.474372,190.896559,150.822418,124.925694,87.748585,103.857734,0.000000,169.271326,77.004193,"[0, 11, 7]",9


#### PCA Cluster Values to use in recommender system

In [21]:
from sklearn.preprocessing import StandardScaler
features = ['zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight',
       'nine', 'ten', 'eleven']
# Separating out the features
x = centroid_distance.loc[:, features].values
# Separating out the target
# Standardizing the features
x = StandardScaler().fit_transform(x)

from sklearn.decomposition import PCA
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(x)
CC_PCA = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1'])
CC_PCA = CC_PCA.reset_index()
CC_PCA = CC_PCA.rename(columns={'index':'cluster'})
CC_PCA = CC_PCA.set_index('cluster')

In [22]:
CC_PCA

,principal component 1
cluster,
0,2.156112
1,-2.129807
2,-1.601025
3,-0.012082
4,-0.473975
5,-2.007320
6,-1.960397
7,0.391749
8,-0.931627


#### Saving for later use

In [23]:
CC_PCA.to_pickle('../Pickle_Files/Cluster_Centers_PCA.pkl')